# Function To Pull Yahoo Stock Data Based on Supplied Tickers

In [3]:
%%writefile CS_StockData.py

import requests
from bs4 import BeautifulSoup
from time import sleep
import csv 
import pandas as pd
import numpy as np

def getStockData(tickers):
    '''Pulls data from Yahoo Finance'''

    #############################################################################################
    stockData = {}
    stockNames = {}
    
    for stock in tickers:
        try:
            stock_url = "https://finance.yahoo.com/quote/"+stock+"/history?p="+stock+""
            r = requests.get(stock_url)
            data = r.text
            soup = BeautifulSoup(data)

            contents = []

            for row in soup.find_all('tbody'):    
                for srow in row.find_all('tr'):
                    for content in srow.find_all('td'):
                        if "Dividend" in content.text:
                            contents.pop() #removes prior entry of date of dividend data point and ignores dividend data point
                            continue
                        else:
                            contents.append(content.text)

#############################################################################################

#pulled all data into one long list, but there are 7 columns of data
#list comprehension to break out the full list in groupings by the 7 columns
            adj_contents = [contents[i * 7:(i + 1) * 7] for i in range((len(contents) + 7 - 1) // 7 )]

#the 7 columns of data, decided not to include most in this analysis
            date=[]
            #open_price=[]
            #high_price=[]
            #low_price=[]
            #close_price=[]
            adj_close_price=[]
            #volume = []
            percent_change = []
            normalized_returns = []

#assigning the values to the columns
            for i in range(0, len(adj_contents)): #change to a count for amount of days requested...up to 97
                date.append(adj_contents[i][0])
                #open_price.append(float(adj_contents[i][1]))
                #high_price.append(float(adj_contents[i][2]))
                #low_price.append(float(adj_contents[i][3]))
                #close_price.append(float(adj_contents[i][4]))
                adj_close_price.append(float(adj_contents[i][5].replace(",","")))
                #volume.append(float(adj_contents[i][6].replace(",","")))

#calculating the percent change from prior close to today        
            for j in range(0, len(adj_close_price)):
                if j == len(adj_close_price)-1:
                    percent_change.append(0.00)
                else:
                    percent_change.append(round(adj_close_price[j]/adj_close_price[j+1]-1,3))

#normalizing returns for comparison across companies
            normalizer = 100

            for x in range(len(percent_change), 0,-1):
                if x == len(percent_change):
                    normalized_returns.append(normalizer)
                else:
                    normalizer = round(normalizer * (1+percent_change[x-1]),3)
                    normalized_returns.append(normalizer)

            normalized_returns.reverse()

#############################################################################################

#placing the values into a panda data frame
            x = pd.DataFrame({"Date": date, 
                              "Adj Close Price": adj_close_price,
                              "Percent Change": percent_change,
                              "Normalized Returns": normalized_returns})
            x["Date"] = pd.to_datetime(x["Date"]) #allows interpretation as dates vs. ordered numbers
#placing data frame into dictionary with stock ticker as key
            stockData[stock] = x

#############################################################################################        

#list obtained from https://dumbstockapi.com/
            stockList = pd.read_csv("StockInformationList.csv")
        
            for y in range(0, len(stocklist)):
                if stock == stocklist["ticker"][y]:
                    stockNames[stock]= stockList["name"][y]     

#############################################################################################  
        except:
            continue
            #stockErrors.append(stock)
            
#delay added to avoid triggering web scrapping blocks
        delay = [0.1, 0.5, 1, 2, 2.5]
        delay = np.random.choice(delay)

        sleep(delay)
    
    return stockNames, stockData


Overwriting CS_StockData.py
